In [1]:
from utils import DateSet, numpy2gpu,load_split_data
from utils import gen_tests_of_deblur,x_post_sample_modify,prior_mean_cov
from utils import sample_cumulated_sum
from function import get_x_ml,W_matrx_1,W_matrx_2
from visualize import printdict
%matplotlib inline

In [2]:
%run problem_setting.py

sigma: [0.001, 0.005, 0.01]
noise_num: 3
M_samples_per_para: 50000
x_dim: 26
data_dim: 50
    H: (50, 26)
    W: (26, 26)
data_file_prefix: data\heat_conduction_mu3_[0.001, 0.005, 0.01]_M50000
---------------------------------------------------------------------------
number of samples: 3*  50000  (noise_num*M_samples_per_para)
---------------------------------------------------------------------------


# load model and simulate test data 

In [3]:
class VAE(nn.Module):

    def __init__(self, layer_sizes, encoder_layer_sizes, latent_size, decoder_layer_sizes,
                 conditional=True, num_labels=0):
        super().__init__()

        if conditional:
            assert num_labels > 0

        assert type(encoder_layer_sizes) == list
        assert type(layer_sizes) == list
        assert type(latent_size) == int
        assert type(decoder_layer_sizes) == list

        self.latent_size = latent_size
        self.net = NN(layer_sizes)
        self.encoder = Encoder(
            encoder_layer_sizes, latent_size, conditional, num_labels)
        self.decoder = Decoder(
            decoder_layer_sizes, latent_size, conditional, num_labels)

    def forward(self, x, unkown, data):
        data = self.net(data)
        class_data = torch.max(data, 1).indices.float().reshape(-1, 1)
        class_data = class_data @ torch.ones(1, 50)
        c = torch.cat([unkown, class_data], dim=1)
        means, log_var = self.encoder(x, c)
        z = self.reparameterize(means, log_var)
        recon_x = self.decoder(z, c)

        return recon_x, z, data, class_data

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)

        return mu + eps * std

    def inference(self, z, unkown, data):
        data = self.net(data)
        class_data = torch.max(data, 1).indices.float().reshape(-1, 1)
        class_data = class_data @ torch.ones(1, 50)
        c = torch.cat([unkown, class_data], dim=1)
        recon_x = self.decoder(z, c)
        return recon_x,data,class_data


class Encoder(nn.Module):

    def __init__(self, layer_sizes, latent_size, conditional, num_labels):

        super().__init__()

        self.conditional = conditional
        if self.conditional:
            layer_sizes[0] += num_labels

        self.MLP = nn.Sequential()

        for i, (in_size, out_size) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
            self.MLP.add_module(name="L{:d}".format(i), module=nn.Linear(in_size, out_size))

            self.MLP.add_module(name="A{:d}".format(i), module=nn.LeakyReLU())

        self.linear_means = nn.Linear(layer_sizes[-1], latent_size)
        self.linear_log_var = nn.Linear(layer_sizes[-1], latent_size)

    def forward(self, x, c=None):

        if self.conditional:
            x = torch.cat((x, c), dim=-1)

        x = self.MLP(x)

        means = self.linear_means(x)
        log_vars = self.linear_log_var(x)

        return means, log_vars


class Decoder(nn.Module):

    def __init__(self, layer_sizes, latent_size, conditional, num_labels):

        super().__init__()

        self.MLP = nn.Sequential()
        self.last_par = torch.nn.parameter.Parameter(torch.normal(0, 0.1, size=(4, 1)))

        self.conditional = conditional
        if self.conditional:
            input_size = latent_size + num_labels
        else:
            input_size = latent_size

        for i, (in_size, out_size) in enumerate(zip([input_size] + layer_sizes[:-1], layer_sizes)):
            self.MLP.add_module(name="L{:d}".format(i), module=nn.Linear(in_size, out_size))
            if i + 2 < len(layer_sizes):
                # self.MLP.add_module(name="A{:d}".format(i), module=nn.BatchNorm1d(out_size))
                self.MLP.add_module(name="A{:d}".format(i), module=nn.LeakyReLU())
            # else:
            #     self.MLP.add_module(name="Output", module=nn.Softplus())#[batchsize,layer_sizes[-1]]

    # 最后一层不可以是1了...只能是指定的类别
    def forward(self, z, c):

        if self.conditional:
            z = torch.cat((z, c), dim=1)
        x = self.MLP(z)
        out = x
        return out


class NN(nn.Module):
    def __init__(self, layer_size):
        super(NN, self).__init__()  # 调用父类的初始化函数
        assert type(layer_size) == list
        self.classMLP = nn.Sequential()
        for i, (in_size, out_size) in enumerate(zip(layer_size[:-1], layer_size[1:])):
            self.classMLP.add_module(name="L{:d}".format(i), module=nn.Linear(in_size, out_size))
            if i + 2 < len(layer_size):
                self.classMLP.add_module(name="A{:d}".format(i), module=nn.ReLU())

    def forward(self, data):
        data = self.classMLP(data)
        data = torch.nn.functional.softmax(data,dim=1)
        return data

In [4]:
def x_post_sample_modify(W,H,m,sigma,y,alpha):
    sigma = sigma + 1e-8
    lamda = 2*alpha/(sigma**2)
#     T_pr_inv = np.linalg.inv(W/lamda)
    T_pr_inv = W * lamda 
    post_cov = np.linalg.inv((1/sigma**2)* H.T@H + T_pr_inv)
    post_mean = post_cov@(H.T@y* (1/(sigma))**2)
    
    post_cov_L = np.linalg.cholesky(post_cov)
    x_post_sample = post_mean+post_cov_L@np.random.randn(m,1)
    return x_post_sample,post_mean,post_cov

In [5]:
#只分3类 
model_file_name = '0506_ls5_cvae_lr9.599999999999998e-05' #?? 0.01 0.0076 0.0027
# 分为5类 model_file_name = '0505_cvae_lr0.0005500000000000001_accuracy0.609375' 
# 分成8 类 
# model_file_name ='0512_cvae_lr0.0011499999999999998_accuracy0.6875'#'0510_cvae_lr0.004600000000000001_accuracy0.7578125'#'0510_cvae_lr0.0051_accuracy0.6484375'0.008,0.008,0.004 #'0510_cvae_lr0.0006000000000000002_accuracy0.6640625'##'0510_cvae_lr0.004600000000000001_accuracy0.7578125'#''
args = np.load(os.path.join('saved_model',model_file_name)+"_args.npy",allow_pickle=True).item()
hypers = np.load(os.path.join('saved_model',model_file_name)+"_hypers.npy",allow_pickle=True).item()

print(model_file_name)
printdict(args),printdict(hypers)

cvae = torch.load(os.path.join('saved_model',model_file_name)+".pth") #??
cvae =  cvae.to('cpu')

0506_ls5_cvae_lr9.599999999999998e-05
epochs: 20
batch_size: 128
layer_sizes: [50, 100, 200, 100, 50, 30, 3]
encoder_layer_sizes: [77, 30, 12, 5]
decoder_layer_sizes: [40, 20, 10, 1]
latent_size: 5
print_every: 5000
fig_root: figs
conditional: True
sigma: [0.001, 0.005, 0.01]
noise_num: 3
M_samples_per_para: 50000
x_dim: 26
data_dim: 50
    H: (50, 26)
    W: (26, 26)
data_file_prefix: heat_conduction_mu3_M50000


$\sigma_{obs} = 0.01$

In [33]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     torch.backends.cudnn.deterministic = True

setup_seed(8)

In [34]:
import time 
N = 10000
y_true = np.array(pd.read_csv('y_true.csv'))
use_sample_size = 5000
x_true = np.array(pd.read_csv('qt_true.csv',header = None))

H = hypers['H']
H_arr = np.array(H)
W = W_matrx_1()
alpha = 2.5e-2
m = hypers['x_dim']


y= np.array(pd.read_csv('tradition100000_0.01.csv',header=None))
x_traditional = np.array(pd.read_csv('y100000_0.01.csv',header=None))
sigma_noise = 0.01

theta_truth = np.zeros(1)#??

x_0 =  get_x_ml(y,hypers)
x_init_value = x_0
# np.random.seed(7)    
x_sum,x_square_sum = np.zeros_like(x_true),np.zeros_like(x_true)
theta_sum,theta_square_sum = np.zeros_like(theta_truth),np.zeros_like(theta_truth)  

THETA= []
X = []

for i in tqdm(range(N)):
    # sample theta
    c = torch.cat((torch.tensor(x_0).float(),torch.tensor(y).float()),dim=0).T
    z = torch.randn([c.size(0), args['latent_size']])
    theta_0,_,classes = cvae.inference(z,torch.tensor(x_0.T).float(),torch.tensor(y.T).float())
    # theta_0,data = cvae.inference(z,torch.tensor(x_0.T).float(),torch.tensor(y.T).float()) #??
    theta_0 = theta_0.detach().numpy()
    sigma = theta_0

    
    x_0,_,_ = x_post_sample_modify(W,H_arr,m,sigma,y,alpha)
    if i>(N-use_sample_size-1):
        theta_sum,theta_square_sum = sample_cumulated_sum(theta_sum,theta_square_sum,theta_0)
        x_sum,x_square_sum = sample_cumulated_sum(x_sum,x_square_sum,x_0)   
    THETA.append(sigma[0])
    X.append(x_0)

x_mean = x_sum/use_sample_size
x_var = x_square_sum/use_sample_size-x_mean**2

theta_mean = theta_sum/use_sample_size
theta_var = theta_square_sum/use_sample_size-theta_mean**2

print(f'sigma_true:{sigma_noise},sigma_mean:{theta_mean},sigma_var:{np.sqrt(theta_var)}')

x = np.linspace(0, 1, m)
interval0 = [1 if (i < 0.4) else 0 for i in x]
interval1 = [1 if (i >= 0.4 and i < 0.8) else 0 for i in x]
interval2 = [1 if (i >= 0.8) else 0 for i in x]
y = (2.5 * x) * interval0 + (2 - 2.5 * x) * interval1 + np.array([0]*m) * interval2

print(np.sqrt(np.sum((x_mean.flatten()-y)**2)/x.shape[0]),np.sqrt(np.sum((x_traditional.flatten()-y)**2)/x.shape[0]))######################3！！！！！！！！！！！！！！！1


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:05<00:00, 1977.05it/s]

sigma_true:0.01,sigma_mean:[[0.01002861]],sigma_var:[[0.00061068]]
0.041445736177933616 0.04161387022752196


In [35]:
import os
path = 'D:\LiulanqiDownload\pycharm\code\heat_conduction_\metroplis-within-gibbs\data821'
np.save(os.path.join(path,f'CwG_xmean_{sigma_noise}.npy'),x_mean)
np.save(os.path.join(path,f'CwG_xvar_{sigma_noise}.npy'),x_var)
np.save(os.path.join(path,f'CwG_theta_{sigma_noise}.npy'), np.array(THETA))
np.save(os.path.join(path,f'CwG_x_{sigma_noise}.npy'), np.array(X))

$\sigma_{obs} = 0.005$

In [ ]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     np.random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(4)

In [17]:
N = 10000
y_true = np.array(pd.read_csv('y_true.csv'))
use_sample_size = 5000
x_true = np.array(pd.read_csv('qt_true.csv',header = None))

H = hypers['H']
H_arr = np.array(H)
W = W_matrx_1()
alpha = 1e-2
m = hypers['x_dim']


# y= np.array(pd.read_csv('tradition100000_0.005.csv',header=None))
y = np.array(pd.read_csv('tradition100000_0.005_test.csv',header=None)).T
x_traditional_005 = np.array(pd.read_csv('y100000_0.005.csv',header=None))
sigma_noise = 0.005

theta_truth = np.zeros(1)#??

x_0 =  get_x_ml(y,hypers)
x_init_value = x_0
np.random.seed(7)    
x_sum,x_square_sum = np.zeros_like(x_true),np.zeros_like(x_true)
theta_sum,theta_square_sum = np.zeros_like(theta_truth),np.zeros_like(theta_truth)  

THETA= []
X = []
start = time.time()
for i in tqdm(range(N)):
    # sample theta
    c = torch.cat((torch.tensor(x_0).float(),torch.tensor(y).float()),dim=0).T
    z = torch.randn([c.size(0), args['latent_size']])
#     theta_0 = cvae.inference(z, c=c).detach().numpy()
#     sigma = theta_0
    # theta_0,data = cvae.inference(z,torch.tensor(x_0.T).float(),torch.tensor(y.T).float())
    theta_0,_,classes = cvae.inference(z,torch.tensor(x_0.T).float(),torch.tensor(y.T).float())
    theta_0 = theta_0.detach().numpy()
    sigma = theta_0

    x_0,_,_ = x_post_sample_modify(W,H_arr,m,sigma,y,alpha)
    if i>(N-use_sample_size-1):
        theta_sum,theta_square_sum = sample_cumulated_sum(theta_sum,theta_square_sum,theta_0)
        x_sum,x_square_sum = sample_cumulated_sum(x_sum,x_square_sum,x_0)   
    THETA.append(sigma[0])
    X.append(x_0)

x_mean = x_sum/use_sample_size
x_var = x_square_sum/use_sample_size-x_mean**2

theta_mean = theta_sum/use_sample_size
theta_var = theta_square_sum/use_sample_size-theta_mean**2
end = time.time()

print('运行时间 : %s 秒' % (end - start))
print(f'sigma_true:{sigma_noise},sigma_mean:{theta_mean},sigma_std:{np.sqrt(theta_var)}')

x = np.linspace(0, 1, m)
interval0 = [1 if (i < 0.4) else 0 for i in x]
interval1 = [1 if (i >= 0.4 and i < 0.8) else 0 for i in x]
interval2 = [1 if (i >= 0.8) else 0 for i in x]
y = (2.5 * x) * interval0 + (2 - 2.5 * x) * interval1 + np.array([0]*m) * interval2

print(np.sqrt(np.sum((x_mean.flatten()-y)**2)/x.shape[0]),np.sqrt(np.sum((x_traditional_005.flatten()-y)**2)/x.shape[0]))

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2164.33it/s]

运行时间 : 4.622421979904175 秒
sigma_true:0.005,sigma_mean:[[0.00766732]],sigma_std:[[0.00067341]]
0.030785173406453502 0.03476813655322245


In [18]:
import os
path = 'D:\LiulanqiDownload\pycharm\code\heat_conduction_\metroplis-within-gibbs\data821'
np.save(os.path.join(path,f'CwG_xmean_{sigma_noise}.npy'),x_mean)
np.save(os.path.join(path,f'CwG_xvar_{sigma_noise}.npy'),x_var)
np.save(os.path.join(path,f'CwG_theta_{sigma_noise}.npy'), np.array(THETA))
np.save(os.path.join(path,f'CwG_x_{sigma_noise}.npy'), np.array(X))

$\sigma_{obs} = 0.001$

In [25]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     np.random.seed(seed)
     torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(4)

In [26]:
N = 10000
y_true = np.array(pd.read_csv('y_true.csv'))
use_sample_size = 5000
x_true = np.array(pd.read_csv('qt_true.csv',header = None))

H = hypers['H']
H_arr = np.array(H)
W = W_matrx_1()
alpha = 5e-4
m = hypers['x_dim']


y= np.array(pd.read_csv('tradition100000_0.001.csv',header=None))
x_traditional_001 = np.array(pd.read_csv('y100000_0.001.csv',header=None))
sigma_noise = 0.001

theta_truth = np.zeros(1)#??

x_0 =  get_x_ml(y,hypers)
x_init_value = x_0
np.random.seed(7)    
x_sum,x_square_sum = np.zeros_like(x_true),np.zeros_like(x_true)
theta_sum,theta_square_sum = np.zeros_like(theta_truth),np.zeros_like(theta_truth)  

THETA= []
X = []
start = time.time()
for i in tqdm(range(N)):
    # sample theta
    c = torch.cat((torch.tensor(x_0).float(),torch.tensor(y).float()),dim=0).T
    z = torch.randn([c.size(0), args['latent_size']])
#     theta_0 = cvae.inference(z, x_0,y).detach().numpy()
#     sigma = theta_0
# #     print(c,z,sigma)
    # theta_0,classes = cvae.inference(z,torch.tensor(x_0.T).float(),torch.tensor(y.T).float())
    theta_0,_,classes = cvae.inference(z,torch.tensor(x_0.T).float(),torch.tensor(y.T).float())
    theta_0 = theta_0.detach().numpy()
    sigma = theta_0


    x_0,_,_ = x_post_sample_modify(W,H_arr,m,sigma,y,alpha)
    if i>(N-use_sample_size):
        theta_sum,theta_square_sum = sample_cumulated_sum(theta_sum,theta_square_sum,theta_0)
        x_sum,x_square_sum = sample_cumulated_sum(x_sum,x_square_sum,x_0)   
    THETA.append(sigma[0])
    X.append(x_0)

x_mean = x_sum/use_sample_size
x_var = x_square_sum/use_sample_size-x_mean**2

theta_mean = theta_sum/use_sample_size
theta_var = theta_square_sum/use_sample_size-theta_mean**2
end = time.time()

print('运行时间 : %s 秒' % (end - start))
print(f'sigma_true:{sigma_noise},sigma_mean:{theta_mean},sigma_std:{np.sqrt(theta_var)}')

x = np.linspace(0, 1, m)
interval0 = [1 if (i < 0.4) else 0 for i in x]
interval1 = [1 if (i >= 0.4 and i < 0.8) else 0 for i in x]
interval2 = [1 if (i >= 0.8) else 0 for i in x]
y = (2.5 * x) * interval0 + (2 - 2.5 * x) * interval1 + np.array([0]*m) * interval2

print(np.sqrt(np.sum((x_mean.flatten()-y)**2)/x.shape[0]),np.sqrt(np.sum((x_traditional_001.flatten()-y)**2)/x.shape[0]))

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2177.11it/s]

运行时间 : 4.595203638076782 秒
sigma_true:0.001,sigma_mean:[[0.00277777]],sigma_std:[[0.00030574]]
0.015024912106400312 0.01491136089527157


In [20]:
import os
path = 'D:\LiulanqiDownload\pycharm\code\heat_conduction_\metroplis-within-gibbs\data821'
np.save(os.path.join(path,f'CwG_xmean_{sigma_noise}.npy'),x_mean)
np.save(os.path.join(path,f'CwG_xvar_{sigma_noise}.npy'),x_var)
np.save(os.path.join(path,f'CwG_theta_{sigma_noise}.npy'), np.array(THETA))
np.save(os.path.join(path,f'CwG_x_{sigma_noise}.npy'), np.array(X))